In [ ]:
%cd ..
%load_ext kedro.ipython

In [ ]:
from raccoon_spotter.utils.data_visualization import *
from raccoon_spotter.utils.metrics import *

In [ ]:
X, Y = catalog.load("raccoon_features_data_array").values(); Y = Y.astype(np.float32)
pretrained_model = catalog.load("pretrained_model")
trained_model = catalog.load("trained_model")

In [ ]:
layer_means = {layer.name: layer.weights[0].numpy().flatten() for layer in pretrained_model.layers if "conv" in layer.name or "dense" in layer.name}
kdeplot(layer_means)
plt.show()

In [ ]:
P0 = pretrained_model.predict(X)
P1 = trained_model.predict(X)

In [ ]:
mse0 = mse(P0, Y)
mse1 = mse(P1, Y)
cos0 = 1+cos(P0, Y)
cos1 = 1+cos(P1, Y)
iou0 = 1-iou(P0, Y)
iou1 = 1-iou(P1, Y)

In [ ]:
# normalize the metrics for comparison
scaled = lambda array: array / max(array)
mse0, mse1 = scaled(np.array([mse0, mse1]))
cos0, cos1 = scaled(np.array([cos0, cos1]))
iou0, iou1 = scaled(np.array([iou0, iou1]))

In [ ]:
radialplot(
	["Intersection\nOver\nUnion\nComplement", "\nMean\nSquared\nError", "\nCosine\nSimilarity"],
	dict(
		pretrained_model=[iou0, mse0, cos0],
		trained_model=[iou1, mse1, cos1]
	),
	intervals = 9
)